In [98]:
# One Dimensional Time Series Data - Regularity Measure of 'Fake News' Sequence
# Math 448, UBC 2019

import numpy as np
import pandas as pd

df = pd.ExcelFile('timestamps-fakenews-1-23-2019.xlsx').parse('Sheet1');
df = df.as_matrix()

ts = np.zeros(len(df))
for i in range(len(df)):
    ts[i] = float(df[i])
    if i-1 >= 0 and df[i-1] == df[i]:
        ts[i-1] = 'NaN'

# Filter out repeated timestamps
timestamps = np.zeros(0)
for i in range(len(ts)):
    if not np.isnan(ts[i]):
        timestamps = np.append(timestamps, ts[i])

shift = timestamps[0]
for i in range(len(timestamps)):
    timestamps[i] = timestamps[i] - shift

In [99]:
timestamps

array([  0.00000000e+00,   1.45653720e+16,   2.20114910e+16, ...,
         2.48424170e+16,   2.48424180e+16,   2.48424190e+16])

In [102]:
n = len(timestamps)

differences = np.zeros(n - 1)

j = 0 # index of differences
for i in range(n):
    if i+1 < n:
        differences[j] = timestamps[i+1] - timestamps[i]
        j += 1
        
differences

array([  1.45653720e+16,   7.44611900e+15,   2.85763000e+14, ...,
         1.00000000e+09,   1.00000000e+09,   1.00000000e+09])

In [118]:
# period = argmin f(p) = sum_{1 to n-1}[|differences[i] - p|]
# optimization problem can be formulated as a linear program

from scipy.optimize import linprog

c = np.ones(n)
c[n-1] = 0

b = np.zeros(2*(n-1))
for i in range(n-1):
    b[2*i] = -differences[i]
    b[2*i+1] = differences[i]

A = np.zeros((2*(n-1), n))
for i in range(n-1):
    A[2*i, i] = -1
    A[2*i, n-1] = -1
    A[2*i+1, i] = -1
    A[2*i+1, n-1] = 1
    
res = linprog(c, A_ub=A, b_ub=b, options=dict(bland=True))

ValueError: Invalid input for linprog with method = 'simplex'.  Number of columns in A_ub must be equal to the size of c

In [121]:
A

array([[-1.,  0.,  0., ...,  0.,  0., -1.],
       [-1.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0., -1.,  0., ...,  0.,  0., -1.],
       ..., 
       [ 0.,  0.,  0., ..., -1.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0., -1., -1.],
       [ 0.,  0.,  0., ...,  0., -1.,  1.]])

In [132]:
error = 0
period = 1e9

for i in range(len(differences)):
    error += abs(period-differences[i])
error

24840468000000000.0

In [137]:
def median(lst):
    n = len(lst)
    if n < 1:
            return None
    if n % 2 == 1:
            return sorted(lst)[n//2]
    else:
            return sum(sorted(lst)[n//2-1:n//2+1])/2.0

period = median(differences)

In [138]:
np.exp(-np.power(error, 2.) / (2*n * np.power(period, 2.)))

0.0